# Subsequent Memory Effect

### Imports

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [3]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [4]:
# load aggregted spectral parameterization results
data = pd.read_csv(PROJECT_PATH + 'data/results/' + 'df_ols_allchans.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)
# data = pd.read_csv(PROJECT_PATH + 'data/results/' + 'df_ols.csv')
print(data.shape)
print(len(data)/4)
data.head()

(2780, 19)
695.0


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,exp_diff,alpha_pre,alpha_post,alpha_diff,peak_present,f_rot,alpha_bp_diff,material,memory,ap_mode
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,-0.037759,0.213201,0.123599,-0.089602,3.0,10.978558,-1536.029206,words,hit,knee
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,-0.218467,0.210785,0.225154,0.014369,3.0,83.836257,-212.749318,words,hit,knee
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,-0.048501,0.276397,NaN,NaN,1.0,89.824561,-2992.875343,words,hit,knee
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,2.940543,2.586441,-0.354102,NaN,0.213641,NaN,2.0,59.883041,-1776.054670,words,hit,knee
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,-0.315757,0.227912,0.182313,-0.045599,3.0,65.871345,-4302.884091,words,hit,knee


### Run 2x2 Anova

In [5]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exp', 'alpha', 'alpha_bp']:
    # Convert NaNs to 0s
    df = data.copy()
    df.loc[np.isnan(df[f'{var}_diff']), f'{var}_diff'] = 0

    # repeated measures anova
    results_i = sm.stats.AnovaRM(df, f"{var}_diff", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.9523 1.0000 694.0000 0.3295
material         4.0674 1.0000 694.0000 0.0441
memory:material  0.0757 1.0000 694.0000 0.7833



===================  alpha  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.0008 1.0000 694.0000 0.9781
material         0.3503 1.0000 694.0000 0.5541
memory:material  0.3869 1.0000 694.0000 0.5342



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.8071 1.0000 694.0000 0.3693
material         4.4911 1.0000 694.0000 0.0344
memory:material  0.0000 1.0000 694.0000 0.9956



Original manuscript applied 2x2 at the level of individual electrodes. This will require further modification of the pipeline

### Restrict analysis to channels exhibiting significant modulation of alpha/beta bandpower

In [6]:
# load task-modulation results
fname_in = r"C:\Users\micha\projects\oscillation_vs_exponent\data\results\ieeg_modulated_channels.csv"
df_tm = pd.read_csv(fname_in)

# modify ccolunm names for consistancy between datframes
df_tm.drop(columns=['Unnamed: 0'], inplace=True)
df_tm.rename(columns={'pval' : 'pval_tm', 'sign' : 'sign_tm'}, inplace=True)
# for mat in ['words','faces']:
#     df_tm.loc[df_tm['material'] == mat, 'material'] = mat[:-1]

# determine significance
df_tm['sig_tm'] = np.where(df_tm['pval_tm'] < 0.05, 1, 0)

# join
df_merge = pd.merge(data, df_tm, on=['patient','chan_idx','material','memory'])
df_merge

df_merge

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,exp_diff,...,material,memory,ap_mode,pval_uncorrected,sign_tm,missing,drop,pval_tm,sig,sig_tm
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,-0.037759,...,words,hit,knee,0.0360,1.0,False,False,0.0360,True,1
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,-0.218467,...,words,hit,knee,0.0280,1.0,False,False,0.0280,True,1
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,-0.048501,...,words,hit,knee,0.0002,1.0,False,False,0.0002,True,1
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,2.940543,2.586441,-0.354102,...,words,hit,knee,0.0014,1.0,False,False,0.0014,True,1
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,-0.315757,...,words,hit,knee,0.0000,1.0,False,False,0.0000,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,2.542326,2.483833,-0.058494,...,faces,miss,knee,0.1656,-1.0,False,False,0.1656,False,0
2776,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,3.030214,2.805684,-0.224530,...,faces,miss,knee,0.9758,-1.0,False,False,0.9758,False,0
2777,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,3.083662,3.047577,-0.036085,...,faces,miss,knee,0.0140,-1.0,False,False,0.0140,True,1
2778,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,2.978981,2.772607,-0.206374,...,faces,miss,knee,0.0368,-1.0,False,False,0.0368,True,1


In [7]:
# find channels that are significant across material conditions
df_hit = df_merge[df_merge['memory']=='hit']
sig = df_hit.groupby(['patient','chan_idx']).all().reset_index()

# add to df_merge
df_merge['sig'] = np.nan
for ii in range(len(sig)):
    df_merge.loc[(df_merge['patient']==sig.loc[ii, 'patient']) & \
                (df_merge['chan_idx']==sig.loc[ii, 'chan_idx']), 'sig'] \
                    = sig.loc[ii, 'sig_tm']
    
# reduce
df_sig = df_merge[df_merge['sig']==1]

# show
print(len(df_sig)/4)
df_sig



145.0


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,exp_diff,...,material,memory,ap_mode,pval_uncorrected,sign_tm,missing,drop,pval_tm,sig,sig_tm
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,-0.037759,...,words,hit,knee,0.0360,1.0,False,False,0.0360,True,1
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,3.052480,2.834013,-0.218467,...,words,hit,knee,0.0280,1.0,False,False,0.0280,True,1
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,2.774045,2.725544,-0.048501,...,words,hit,knee,0.0002,1.0,False,False,0.0002,True,1
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,2.872949,2.557193,-0.315757,...,words,hit,knee,0.0000,1.0,False,False,0.0000,True,1
9,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,pat04_0,3.189699,2.446979,-0.742720,...,words,hit,knee,0.0000,1.0,False,False,0.0000,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,pat22,25,PI_05-PI_06,-10.50000,-40.00000,32.50000,pat22_25,4.165989,4.157912,-0.008077,...,faces,miss,knee,0.0302,-1.0,False,False,0.0302,True,1
2753,pat22,26,PI_06-PI_07,-11.50000,-40.00000,39.00000,pat22_26,3.771881,3.858311,0.086430,...,faces,miss,knee,0.1644,-1.0,False,False,0.1644,True,0
2756,pat22,29,PI_09-PI_10,-11.50000,-41.00000,60.50000,pat22_29,4.296499,4.842026,0.545527,...,faces,miss,knee,0.1330,-1.0,False,False,0.1330,True,0
2763,pat22,36,AC_07-AC_08,23.50000,-31.00000,42.00000,pat22_36,3.241910,3.060999,-0.180912,...,faces,miss,knee,0.0052,-1.0,False,False,0.0052,True,1


In [8]:
# NaN handling - identify channels with no detectable alpha peak

# take mean for each patient/channel across conditions to find NaNs
df_nan = df_sig.groupby(['patient','chan_idx']).mean()['alpha_diff'].reset_index()
df_nan['nan'] = np.where(np.isnan(df_nan['alpha_diff']), 1, 0)

# add to df_sig
df_sig['alpha_diff_nan'] = False
for ii in range(len(df_nan)):
    df_i = df_sig.loc[(df_sig['patient']==df_nan.loc[ii, 'patient']) & \
                        (df_sig['chan_idx']==df_nan.loc[ii, 'chan_idx'])]
    if np.isnan(df_i['alpha_diff']).any():
        df_sig.loc[(df_sig['patient']==df_nan.loc[ii, 'patient']) & \
                    (df_sig['chan_idx']==df_nan.loc[ii, 'chan_idx']), 'alpha_diff_nan'] = True
    

# reduce
df_alpha = df_sig.loc[df_sig['alpha_diff_nan']==False]

# show
print(np.sum(np.isnan((df_alpha['alpha_diff']))))
df_alpha.head()


C:\Users\micha\AppData\Local\Temp\ipykernel_62652\1329950098.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sig['alpha_diff_nan'] = False


0


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,exp_pre,exp_post,exp_diff,...,memory,ap_mode,pval_uncorrected,sign_tm,missing,drop,pval_tm,sig,sig_tm,alpha_diff_nan
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,3.542673,3.504914,-0.037759,...,hit,knee,0.0360,1.0,False,False,0.0360,True,1,False
9,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,pat04_0,3.189699,2.446979,-0.742720,...,hit,knee,0.0000,1.0,False,False,0.0000,True,1,False
10,pat04,1,A02-A10,-58.82445,57.49465,-20.34855,pat04_1,3.291404,3.083202,-0.208201,...,hit,knee,0.0034,1.0,False,False,0.0034,True,1,False
47,pat04,38,B02-B03,-83.48715,27.97570,43.80840,pat04_38,3.373567,3.281039,-0.092528,...,hit,knee,0.0000,1.0,False,False,0.0000,True,1,False
54,pat04,45,A01-A02,-63.34330,53.12490,-21.48615,pat04_45,3.220435,2.459780,-0.760655,...,hit,knee,0.0000,1.0,False,False,0.0000,True,1,False


In [9]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var, df in zip(['exp', 'alpha', 'alpha_bp'], [df_sig, df_alpha, df_sig]):
    # repeated measures anova
    results_i = sm.stats.AnovaRM(df, f"{var}_diff", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           5.1089 1.0000 144.0000 0.0253
material         6.9942 1.0000 144.0000 0.0091
memory:material  0.6597 1.0000 144.0000 0.4180



===================  alpha  ===================

                    Anova
                F Value Num DF  Den DF Pr > F
---------------------------------------------
memory           0.0045 1.0000 59.0000 0.9469
material         1.1508 1.0000 59.0000 0.2877
memory:material  0.7378 1.0000 59.0000 0.3938



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.0803 1.0000 144.0000 0.7773
material         0.0198 1.0000 144.0000 0.8883
memory:material  3.3540 1.0000 144.0000 0.0691

